**Question: is it possible to predict the number of followers [followers_count] that a person will have on Twitter, based on their other attributes? **

A user's attributes include:
- When the account was created [created_at]: probably super important. The longer someone has been a user for, likely the more followers they have
- Whether or not they have a profile image [default_profile_image]. I'd expect users with a profile image to have more followers
- Description: Could length of description make a difference?
- favourites_count: More active users could favourite more, and also have more followers?
- following: Could potentially be a positive correlation between followers and following, although not necessarily; some very popular accounts are following very few people
- lang: I'd expect users using english/spanish to have more followers
- statuses_count: people with more statuses are more active users, and probably have more followers
- verified: More likely to be a real human
- withheld_in_countries: users with witheld profile possibly have fewer followers?

How many data points do I need for my model? Some sources state at least 20 observations per vairable as a general rule of thunb. How many user profiles can I pull from Twitter in a reasonable amount of time?


Should I get my users from a particular source, or can I just pull them randomly from Twitter?

In [164]:
import twitterscrape
import tweepy
import time
import pandas as pd

In [165]:
api = twitterscrape.authenticate()

In [166]:
#This gets all of the ids for the followers of the RC
ids = []
for page in tweepy.Cursor(api.followers_ids, screen_name="recursecenter").pages():
    ids.extend(page)
    time.sleep(60)

In [167]:
len(ids)


17548

Twitter places [limits](https://dev.twitter.com/rest/public/rate-limits) on how many times you can ping them per 15 minutes. [GET Users/Lookup](https://dev.twitter.com/rest/reference/get/users/lookup) returns up to 100 users per request. I am allowed to send 180 Lookup requests per 15 minutes = 18000 users per 15 minutes. So I should be able to do them all no problem in one 15 minute block, no token bucket required.

In [168]:
ids[0]

100408795

In [169]:
def get_users(ids):
    last_index = 0
    results = []
    for i in range(100,len(ids),100):
        result = api.lookup_users(ids[last_index:i])
        
        last_index = i
        results.append(result)
    return results

In [170]:
RC_followers = get_users(ids)

In [171]:
len(RC_followers)

175

In [176]:
RC_followers[0][4].name 

u'Stewart Stewart'

In [180]:
RC_followers_concat =  [inner 
                        for outer in RC_followers 
                        for inner in outer]

In [184]:
names = [user.name for user in RC_followers_concat]

In [187]:
RC_followers_df = pd.DataFrame(columns = ["name", "created_at","default_profile_image", "description", "favourites_count", "followers_count", "friends_count", "lang", "listed_count", "protected", "statuses_count", "time_zone", "verified", "withheld_in_countries"]
)


In [209]:
getattr(RC_followers_concat[0],'name')

u'Dave Bryand'

In [204]:
RC_followers_df.append?

,name,created_at,default_profile_image,description,favourites_count,followers_count,friends_count,lang,listed_count,protected,statuses_count,time_zone,verified,withheld_in_countries


In [155]:
one = pd.Series([0,1,2,3])
two = pd.Series([0,1,2,3])
three = one.append(two)

In [211]:
columns_list = ["name", "created_at","default_profile_image", "description", "favourites_count", "followers_count", "friends_count", "lang", "listed_count", "protected", "statuses_count", "time_zone", "verified", "withheld_in_countries"]


In [214]:
user_dict = {column:[getattr(user,column) if hasattr(user,column) else None for user in RC_followers_concat] for column in columns_list}
RC_followers_df = pd.DataFrame(user_dict)

In [216]:
len(RC_followers_df)

17500

In [191]:
for column in RC_followers_df.columns:
        print RC_followers_df[column]

Series([], Name: name, dtype: object)
Series([], Name: created_at, dtype: object)
Series([], Name: default_profile_image, dtype: object)
Series([], Name: description, dtype: object)
Series([], Name: favourites_count, dtype: object)
Series([], Name: followers_count, dtype: object)
Series([], Name: friends_count, dtype: object)
Series([], Name: lang, dtype: object)
Series([], Name: listed_count, dtype: object)
Series([], Name: protected, dtype: object)
Series([], Name: statuses_count, dtype: object)
Series([], Name: time_zone, dtype: object)
Series([], Name: verified, dtype: object)
Series([], Name: withheld_in_countries, dtype: object)


In [143]:
RC_followers_df

,name,created_at,default_profile_image,description,favourites_count,followers_count,friends_count,lang,listed_count,protected,statuses_count,time_zone,verified,withheld_in_countries
0,Martin Dupont,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,The Software Guild,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,☻☺şľăᶌịĈỼ卐ĎŏĎð☺☻,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Bruno Santos,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,chq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Anathem Wisenheimer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Antonio Vera,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Andrea Heyman,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Piotr Ujazdowski,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,corback(),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [126]:
stuff = pd.Series([0,1,2,3])
s = RC_followers_df.name.append(stuff)
s 

0    0
1    1
2    2
3    3
dtype: int64

In [131]:
names = []
for i in range(len(RC_followers)):
    for column in RC_followers_df.columns:
        RC_followers_df[column] = RC_followers_df[column].append(pd.Series([user.name for user in RC_followers[i]]))
        
        #RC_followers_df.name.append(names)

ValueError: cannot reindex from a duplicate axis

In [96]:
RC_followers_df

,name,created_at,default_profile_image,description,favourites_count,followers_count,friends_count,lang,listed_count,protected,statuses_count,time_zone,verified,withheld_in_countries


In [82]:
len(RC_followers)

175

In [23]:
len(test[0])

TypeError: object of type 'User' has no len()